# WhisperTS joining JSON to SRT

In [353]:
import re,codecs,unidecode
import pandas as pd
import networkx as nx
import matplotlib.pylab as p

import json,glob,os
nMachine=os.uname()[1]
if "iMac" in nMachine:
    nMachine="iMac"
elif "MacBook" in nMachine:
    nMachine="MacBook"
print(nMachine)

iMac


In [354]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

In [355]:
repJson="/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/%s/Json/"%nMachine
segments=len(glob.glob(repJson+"*.json"))
segments

5

In [356]:
json1=repJson+"extrait-00.json"
with open(json1) as inFile:
    whisper1=json.load(inFile)

### Gestion des timestamps

In [357]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

def shiftTime(timestamp,decal):
    return second2timestamp(timestamp2second(timestamp)+decal)

In [358]:
punct = list('''!()[]{};:-'"\,<>./?@#$%^&*_~''')
maxSize=8

def removePunct(texte):
    result=texte
    for p in punct:
        result=result.replace(p,"")
    return result

### Gestion des structures JSON de Whisper

In [359]:
def getWords(whisper):
    lWords=[]
    for segment in whisper["segments"]:
        for word in segment["words"]:
            lWords.append((second2timestamp(word["start"]),
                              second2timestamp(word["end"]),
                              word["text"],
                              word["confidence"]))        
    return [(t1,t2,w,c) for t1,t2,w,c in lWords if w!=""]

def getList(tWhisper):
    return [unidecode.unidecode(removePunct(w)).lower() for _,_,w,_ in tWhisper]

In [360]:
srt=getWords(whisper1)
srtList=getList(srt)

### Trouver la jointure

In [361]:
def findSample(sample,ref):
    lowSample=[t.lower() for t in sample]
    lowRef=[t.lower() for t in ref]
    i=0
    while i<len(ref)-len(sample):
        # print("ref",i)
        if lowSample==lowRef[i:i+len(sample)]:
            return i
        else:
            i+=1
    return None

### Collage des mots
1. on cherche la partie commune entre le nouveau segment et les sous-titres précédents
2. on peut régler la longueur de la partie commune pour optimiser le collage *maxSize*
3. on incrémente les temps de la nouvelle partie à partir du deuxième mot (pour éviter les erreurs de timestamp en début d'enregistrement)

In [362]:
def pasteWords(srt,whisper2):
    srtList=getList(srt)
    w2=getWords(whisper2)
    w2List=getList(w2)
    lenW2=len(w2)
    compSize=min(maxSize,lenW2)
    i=1
    print(compSize)
    while i<lenW2-compSize:
        sample=w2List[i:i+compSize]
        ref=srtList
        # print(sample)
        idx=findSample(sample,ref)
        if idx:
            print("found",i,idx)
            break
        else:
            i+=1
    if idx:
        print("found",i,idx,srtList[idx],srt[idx][0],w2List[i],w2[i][0])
        tDecal=timestamp2second(srt[idx+1][0])-timestamp2second(w2[i+1][0])
        print(second2timestamp(tDecal))
        newSrt=[]
        for n in range(i+1,lenW2):
            nWord=w2[n]
            newWord=(shiftTime(nWord[0],tDecal),shiftTime(nWord[1],tDecal),nWord[2],nWord[3])
            newSrt.append(newWord)
        print(srt[idx+1],newSrt[0])
        return srt[:idx]+newSrt
    else:
        print("problème")
        ding()
        ding()
        ding()
        return None

In [363]:
if segments:
    for i in range(1,segments):
        print(i)
        json2=repJson+"extrait-%02d.json"%i
        with open(json2) as inFile:
            whisper2=json.load(inFile)
        newSrt=pasteWords(srt,whisper2)
        if not newSrt:
            break
        else:
            srt=newSrt
# print(srt)

1
8
found 1 94
found 1 94 ils 00:01:00,439 ils 00:00:00,320
00:00:59,999
('00:01:00,979', '00:01:01,159', 'ont', 0.986) ('00:01:00,978', '00:01:01,138', 'ont', 0.531)
2
8
found 11 226
found 11 226 une 00:02:05,378 une 00:00:05,139
00:02:00,097
('00:02:05,437', '00:02:05,778', 'mamie', 0.996) ('00:02:05,436', '00:02:05,878', 'mamie', 0.838)
3
8
found 26 365
found 26 365 be 00:03:14,007 be 00:00:13,779
00:03:00,127
('00:03:14,127', '00:03:14,307', 'kind.', 0.898) ('00:03:14,127', '00:03:14,447', 'kind', 0.761)
4
8
found 1 484
found 1 484 on 00:04:03,127 on 00:00:02,560
00:04:00,106
('00:04:03,227', '00:04:03,447', 'va', 0.815) ('00:04:03,227', '00:04:03,406', 'va', 0.914)


In [364]:
pause=0.200

srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=second2timestamp(0)
pEnd=0

for i,(start,end,mot,confidence) in enumerate(srt):
    # print(start,end,mot,confidence)
    tStart=timestamp2second(start)
    tEnd=timestamp2second(end)
    # print(start,tStart,prevEnd)
    if tStart-prevEnd>pause and prevEnd!=0:
        pEnd=second2timestamp(prevEnd)
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=start
        prevEnd=tEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=tEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [365]:
ding()

In [366]:
srtLines=[]
for i,(start,end,mot) in enumerate(srtTours):
    print(start,end,mot)
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


00:00:00,000 00:00:01,720 [*] Alors la première image,
00:00:01,960 00:00:04,000 [*] tu me dis tout ce que tu vois.
00:00:05,019 00:00:06,019 Ils sont dans une classe.
00:00:14,199 00:00:15,038 Il y a des escaliers.
00:00:16,660 00:00:17,620 Il y a des enfants.
00:00:18,280 00:00:19,000 Il y a une maîtresse.
00:00:20,920 00:00:22,539 Il y a un enfant qui a un chewing-gum.
00:00:23,079 00:00:24,559 Il y a un enfant qui a un ballon.
00:00:27,579 00:00:28,920 Il y a un enfant qui a un livre.
00:00:30,000 00:00:35,600 [*] La maîtresse et la dame nette. Il y a des calculs derrière.
00:00:39,039 00:00:40,219 Il y a un avion en [*] papier.
00:00:48,960 00:00:51,539 Il y a plein d'enfants.
00:00:58,159 00:00:59,780 Ils ont l'air comment, les enfants ?
00:01:00,000 00:01:00,439 [*]
00:01:00,978 00:01:02,357 ont l'air de faire tête coquée.
00:01:03,159 00:01:03,457 Ah oui ?
00:01:04,118 00:01:06,456 Et il y en a une qui est triste, on dirait.
00:01:07,878 00:01:08,078 OK.
00:01:09,778 00:01:10,9

In [367]:
with codecs.open(json1.replace(".json","-tours.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")